# Geospatial Matching Optimization

This project is designed to showcase how to improve the speed of a polygon matching procedure. There are two main areas that could benefit from taking a "geospatial-native" approach:

1. Data encoding, fetching, and storage
2. Matching

We will generate synthetic polygon data and use both a non-geospatial-native and geospatial-native approach, comparing execution time for both to see how they differ.

## Imports

In [ ]:
import geopandas as gpd
import pandas as pd
import shapely
import h3
import time
import helpers as h
import matplotlib.pyplot as plt
from shapely.wkt import dumps, loads
from pathlib import Path
import multiprocessing as mp
import datetime

# ensure helpers is loaded correctly
import importlib
importlib.reload(h)

<module 'helpers' from '/Users/sra/files/projects/matching_optimization/helpers.py'>

## Retrieve Generated Test Data

In [1180]:
prev_month_blobs = gpd.read_parquet('data/prev_month_blobs.parquet')
curr_month_blobs = gpd.read_parquet('data/curr_month_blobs.parquet')

In [1181]:
# save to geojson to confirm in QGIS
blobs_geojson = Path('data/prev_month_blobs.geojson')
if not blobs_geojson.is_file():
    prev_month_blobs.to_file('data/prev_month_blobs.geojson')
    curr_month_blobs.to_file('data/curr_month_blobs.geojson')

In [1182]:
prev_month_blobs.head()

,geometry,id,geohash
0,"POLYGON ((-82.05268 27.29791, -82.03920 27.297...",5a109c59-1430-4663-a7a0-cebd76e410bb,dhvekv
1,"POLYGON ((-81.77015 27.80770, -81.68065 27.807...",644ac7ad-375f-47e6-af21-c2035f1f0cd8,dhvyk7
2,"POLYGON ((-82.30067 29.14965, -82.28786 29.149...",b2b0f598-2586-4f30-8a9a-7b809e9d563b,djjmzm
3,"POLYGON ((-81.65674 29.14071, -81.58627 29.140...",0bb5639f-6dc6-46fa-a0e9-9de238a8d813,djjyn9
4,"POLYGON ((-85.35515 30.68264, -85.28704 30.682...",2646987f-a55f-44f9-a7a2-fa4bef0efa44,dj7qdr


In [1183]:
prev_month_blobs = prev_month_blobs[['geometry', 'id', 'geohash']]
prev_month_blobs.head()

,geometry,id,geohash
0,"POLYGON ((-82.05268 27.29791, -82.03920 27.297...",5a109c59-1430-4663-a7a0-cebd76e410bb,dhvekv
1,"POLYGON ((-81.77015 27.80770, -81.68065 27.807...",644ac7ad-375f-47e6-af21-c2035f1f0cd8,dhvyk7
2,"POLYGON ((-82.30067 29.14965, -82.28786 29.149...",b2b0f598-2586-4f30-8a9a-7b809e9d563b,djjmzm
3,"POLYGON ((-81.65674 29.14071, -81.58627 29.140...",0bb5639f-6dc6-46fa-a0e9-9de238a8d813,djjyn9
4,"POLYGON ((-85.35515 30.68264, -85.28704 30.682...",2646987f-a55f-44f9-a7a2-fa4bef0efa44,dj7qdr


In [1184]:
curr_month_blobs = curr_month_blobs[['geometry', 'id', 'geohash']]
curr_month_blobs.head()

,geometry,id,geohash
0,"POLYGON ((-98.30862 27.51270, -98.25725 27.512...",c563b685-22a7-4f9a-8e0b-6a552e5226f7,9ufhg8
1,"POLYGON ((-98.31217 28.77062, -98.21778 28.770...",6c6edb14-00a0-4013-8e47-a09eaccc5c47,9v45gv
2,"POLYGON ((-98.74204 29.03299, -98.72244 29.032...",4f76d8ce-2381-4fd8-9acb-96dae7eac13e,9v1v3m
3,"POLYGON ((-97.06861 29.71133, -97.05833 29.711...",5b8c0722-b61c-44cd-8bbf-4398efa982d1,9v6cp7
4,"POLYGON ((-94.94960 29.99430, -94.93313 29.994...",db0b5ceb-b0d8-4f84-8d00-30e2db9c7337,9vk6zd


In [1185]:
# make data folder
Path('data/').mkdir(exist_ok=True)

# save as geoparquet
prev_month_blobs.to_parquet('data/prev_month_blobs.parquet')
curr_month_blobs.to_parquet('data/curr_month_blobs.parquet')

In [1186]:
# range_ = 10000

# fig, ax = plt.subplots(figsize=(5, 5))
# prev_month_blobs.iloc[0:range_].plot(ax=ax)
# ax.set_title(f'Viewing the first {range_} blobs')
# plt.tight_layout()

# plt.show()

In [1187]:
prev_month_blobs.head()

,geometry,id,geohash
0,"POLYGON ((-82.05268 27.29791, -82.03920 27.297...",5a109c59-1430-4663-a7a0-cebd76e410bb,dhvekv
1,"POLYGON ((-81.77015 27.80770, -81.68065 27.807...",644ac7ad-375f-47e6-af21-c2035f1f0cd8,dhvyk7
2,"POLYGON ((-82.30067 29.14965, -82.28786 29.149...",b2b0f598-2586-4f30-8a9a-7b809e9d563b,djjmzm
3,"POLYGON ((-81.65674 29.14071, -81.58627 29.140...",0bb5639f-6dc6-46fa-a0e9-9de238a8d813,djjyn9
4,"POLYGON ((-85.35515 30.68264, -85.28704 30.682...",2646987f-a55f-44f9-a7a2-fa4bef0efa44,dj7qdr


## 1. Non-optimized process

### 1.1. Data encoding, storage, and fetching

We will use PostgreSQL and string storage for geographic information.

#### Make copy of polygon layer and convert to non-geospatial-native string (WKT) datatype

In [1188]:
# copy the gdfs
prev_month_blobs_wkt = prev_month_blobs.copy()
curr_month_blobs_wkt = curr_month_blobs.copy()

In [1189]:
# Convert WKT versions to strings
dfs_to_convert = [prev_month_blobs_wkt, curr_month_blobs_wkt]
prev_month_blobs_wkt, curr_month_blobs_wkt = [h.convert_col_to_string(df) for df in dfs_to_convert]

# Check result
print(prev_month_blobs_wkt.head())

Function `convert_col_to_string` executed in 0.0948 sec, CPU: 78.20%, Memory: 7.33MB
Function `convert_col_to_string` executed in 0.0208 sec, CPU: 26.20%, Memory: 10.30MB
                                            geometry  \
0  POLYGON ((-82.052682 27.297908, -82.0392 27.29...   
1  POLYGON ((-81.770152 27.807701, -81.680647 27....   
2  POLYGON ((-82.300672 29.149655, -82.287861 29....   
3  POLYGON ((-81.656743 29.140713, -81.586271 29....   
4  POLYGON ((-85.355154 30.682643, -85.287036 30....   

                                     id geohash  
0  5a109c59-1430-4663-a7a0-cebd76e410bb  dhvekv  
1  644ac7ad-375f-47e6-af21-c2035f1f0cd8  dhvyk7  
2  b2b0f598-2586-4f30-8a9a-7b809e9d563b  djjmzm  
3  0bb5639f-6dc6-46fa-a0e9-9de238a8d813  djjyn9  
4  2646987f-a55f-44f9-a7a2-fa4bef0efa44  dj7qdr  


/Users/sra/files/projects/matching_optimization/helpers.py:321: UserWarning: Geometry column does not contain geometry.
  df[col] = df[col].apply(lambda geom: to_wkt(geom) if isinstance(geom, BaseGeometry) else str(geom))
/Users/sra/files/projects/matching_optimization/helpers.py:321: UserWarning: Geometry column does not contain geometry.
  df[col] = df[col].apply(lambda geom: to_wkt(geom) if isinstance(geom, BaseGeometry) else str(geom))


#### Save in PostgreSQL database

This is a simple version of what a "non-geospatial-native" data ingestion pipeline may look like, with the primary example being that the polygons are stored as strings, not in a spatial-friendly datatype.

In [1190]:
# Convert to tuples
prev_month_blobs_wkt = h.df_itertuple(prev_month_blobs_wkt)
curr_month_blobs_wkt = h.df_itertuple(curr_month_blobs_wkt)

Function `df_itertuple` executed in 0.0005 sec, CPU: 50.00%, Memory: 0.02MB
Function `df_itertuple` executed in 0.0004 sec, CPU: 33.30%, Memory: 0.00MB


Create PostgreSQL database if it doesn't exist already. We will be using the default settings. If they need to be adjusted, refer to [`helpers.py`](helpers.py).

In [1191]:
h.create_pg_db()

Database blob_matching already exists.
Function `create_pg_db` executed in 0.0351 sec, CPU: 0.00%, Memory: 0.00MB


Create and insert into tables:

In [1192]:
prev_month_blobs.head(1)

,geometry,id,geohash
0,"POLYGON ((-82.05268 27.29791, -82.03920 27.297...",5a109c59-1430-4663-a7a0-cebd76e410bb,dhvekv


In [1193]:
prev_month_blobs_wkt[0]

('POLYGON ((-82.052682 27.297908, -82.0392 27.297908, -82.0392 27.338549, -82.052682 27.338549, -82.052682 27.297908))',
 '5a109c59-1430-4663-a7a0-cebd76e410bb',
 'dhvekv')

In [1194]:
h.create_pg_table(table_name='prev_blobs_wkt', data=prev_month_blobs_wkt, truncate=True)
h.create_pg_table(table_name='curr_blobs_wkt', data=curr_month_blobs_wkt, truncate=True)

Table prev_blobs_wkt truncated.
Inserted 1000 records into prev_blobs_wkt.
Function `create_pg_table` executed in 0.0653 sec, CPU: 0.00%, Memory: 0.00MB
Table curr_blobs_wkt truncated.
Inserted 1000 records into curr_blobs_wkt.
Function `create_pg_table` executed in 0.0415 sec, CPU: 0.00%, Memory: 0.00MB


Retrieve data as GeoDataFrames to confirm that it worked:

In [1195]:
df_prev = h.retrieve_pg_table(table_name='prev_blobs_wkt')
df_curr = h.retrieve_pg_table(table_name='curr_blobs_wkt')

Retrieved 1000 records from prev_blobs_wkt.
Function `retrieve_pg_table` executed in 0.0099 sec, CPU: 0.00%, Memory: 0.00MB
Retrieved 1000 records from curr_blobs_wkt.
Function `retrieve_pg_table` executed in 0.0071 sec, CPU: 0.00%, Memory: 0.00MB


Compare the tables before and after for a sanity check:

In [1196]:
df_prev.head()

,geometry,id,geohash
0,"POLYGON ((-82.052682 27.297908, -82.0392 27.29...",5a109c59-1430-4663-a7a0-cebd76e410bb,dhvekv
1,"POLYGON ((-81.770152 27.807701, -81.680647 27....",644ac7ad-375f-47e6-af21-c2035f1f0cd8,dhvyk7
2,"POLYGON ((-82.300672 29.149655, -82.287861 29....",b2b0f598-2586-4f30-8a9a-7b809e9d563b,djjmzm
3,"POLYGON ((-81.656743 29.140713, -81.586271 29....",0bb5639f-6dc6-46fa-a0e9-9de238a8d813,djjyn9
4,"POLYGON ((-85.355154 30.682643, -85.287036 30....",2646987f-a55f-44f9-a7a2-fa4bef0efa44,dj7qdr


In [1197]:
prev_month_blobs.head()

,geometry,id,geohash
0,"POLYGON ((-82.05268 27.29791, -82.03920 27.297...",5a109c59-1430-4663-a7a0-cebd76e410bb,dhvekv
1,"POLYGON ((-81.77015 27.80770, -81.68065 27.807...",644ac7ad-375f-47e6-af21-c2035f1f0cd8,dhvyk7
2,"POLYGON ((-82.30067 29.14965, -82.28786 29.149...",b2b0f598-2586-4f30-8a9a-7b809e9d563b,djjmzm
3,"POLYGON ((-81.65674 29.14071, -81.58627 29.140...",0bb5639f-6dc6-46fa-a0e9-9de238a8d813,djjyn9
4,"POLYGON ((-85.35515 30.68264, -85.28704 30.682...",2646987f-a55f-44f9-a7a2-fa4bef0efa44,dj7qdr


Confirm that the original blob dfs and the retrieved dfs are the same:

In [1198]:
def round_geometry(geom, precision=6):
    """Round all coordinates of a geometry to a given precision."""
    return shapely.wkt.loads(shapely.wkt.dumps(geom, rounding_precision=precision))

# Convert both to sets of rounded WKT strings
set_prev_month_blobs = set(prev_month_blobs['geometry'].apply(lambda g: round_geometry(g, precision=6).wkt))
set_df_prev = set(df_prev['geometry'].apply(lambda g: round_geometry(g, precision=6).wkt))

# Find common, missing, and extra geometries
common_geometries = set_prev_month_blobs & set_df_prev
missing_from_retrieved = set_prev_month_blobs - set_df_prev
extra_in_retrieved = set_df_prev - set_prev_month_blobs

# Print summary
print(f"Number of matching geometries: {len(common_geometries)}")
print(f"Missing geometries in retrieved table: {len(missing_from_retrieved)}")
print(f"Extra geometries in retrieved table: {len(extra_in_retrieved)}")

# Show an example missing/extra geometry for debugging
if missing_from_retrieved:
    print("Example missing record:", next(iter(missing_from_retrieved)))

if extra_in_retrieved:
    print("Example extra record:", next(iter(extra_in_retrieved)))

Number of matching geometries: 1000
Missing geometries in retrieved table: 0
Extra geometries in retrieved table: 0


It worked! Now we can proceed to "matching" any "previous" polygons that overlap with the "current" polygons.

### 1.2. Matching

We will match the polygons using GeoPandas.

In [ ]:
print('Start time:', datetime.datetime.now())
postgresql_details = h.pg_details()
h.run_parallel_matching(table_prev='prev_blobs_wkt', 
                        table_curr='curr_blobs_wkt', 
                        output_table='matched_results', 
                        postgresql_details=postgresql_details, 
                        db_name='blob_matching', 
                        num_workers=5,
                        verbose=2)
print('End time:', datetime.datetime.now())

Table matched_results created successfully.
Total batches to process: 1
Dispatched batch 1/1 (100.0% complete)


Process Process-2894:
Traceback (most recent call last):
  File "/Applications/Anaconda/anaconda3/envs/geo/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Applications/Anaconda/anaconda3/envs/geo/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/sra/files/projects/matching_optimization/helpers.py", line 1106, in logging_listener
    record = log_queue.get()
             ^^^^^^^^^^^^^^^
  File "<string>", line 2, in get
  File "/Applications/Anaconda/anaconda3/envs/geo/lib/python3.11/multiprocessing/managers.py", line 822, in _callmethod
    kind, result = conn.recv()
                   ^^^^^^^^^^^
  File "/Applications/Anaconda/anaconda3/envs/geo/lib/python3.11/multiprocessing/connection.py", line 250, in recv
    buf = self._recv_bytes()
          ^^^^^^^^^^^^^^^^^^
  File "/Applications/Anaconda/anaconda3/envs/geo/lib/python3.11/multiprocessing/connection.py", line 430,

KeyboardInterrupt: 

In [ ]:
# # testing logging for matching_geometries
# df_prev = h._retrieve_pg_table(postgresql_details, 'blob_matching', "prev_blobs_wkt")#, log_enabled=True)
# df_curr = h._retrieve_pg_table(postgresql_details, 'blob_matching', "curr_blobs_wkt")#, log_enabled=True)

# # Use a small subset for testing
# # df_prev_sample = df_prev.sample(10, random_state=42)
# # df_curr_sample = df_curr.sample(10, random_state=42)

# # h.match_geometries(df_prev_sample, df_curr_sample)  # Check if this logs matches
# h.match_geometries(df_prev, df_curr)  # Check if this logs matches

TypeError: match_geometries() missing 1 required positional argument: 'log_queue'

In [ ]:
count = h.retrieve_pg_table(table_name='matched_results', count=True)

## 2. Optimized Approach

Having demonstrated a reasonable if not optimized way of matching polygons, we will now make use of geospatial-native tools to hopefully match polygons much more quickly.

My proposed workflow centers around `DuckDB` and `H3` along with the `.parquet` format to efficiently store, retrieve, and process geospatial data. We will use `Apache Iceberg` as a layer over the parquet file as it allows for more robust table handling.

1. Prepare GeodataFrames with H3 Polyfill and parent hexagons at lower resolution.
   * Save as an Iceberg table.

2. Use DuckDB and the H3 extension to execute spatial SQL queries to find overlapping polygons with H3-based joins.

3. Save matching polygon IDs to a new Iceberg table.

4. Visualize the results.

### 2.1. Add H3 data to the blobs

H3 is a modern and efficient geospatial index paradigm that divides the world into nested hexagons.

In [902]:
prev_month_blobs.head()

,geometry,id,geohash
0,"POLYGON ((-81.10504 26.01885, -81.04984 26.018...",6cc95c8a-cbaf-4993-a9ff-4e890c7e45f0,dhwk5h
1,"POLYGON ((-80.91675 26.04190, -80.86358 26.041...",a0fbead2-d8dc-4e05-b364-ab89d020fbfb,dhwkpq
2,"POLYGON ((-80.73525 26.20304, -80.71903 26.203...",b4c64915-35eb-4d74-a211-ce41ad8f21f8,dhwt55
3,"POLYGON ((-80.86288 26.17613, -80.79088 26.176...",d03d2784-ff88-4ed6-b93c-2b52a8a7ecc3,dhwt0s
4,"POLYGON ((-81.28584 26.19397, -81.21442 26.193...",0ce2ca44-0727-4364-978b-da23cd9cd784,dhwjpj


In [903]:
def compute_h3_indices(geometry, centroid_res=6, polyfill_res=9):
    """
    Compute H3 indices for a polygon:
    - Single H3 index based on centroid
    - Full polygon coverage with H3 polyfill at high resolution
    """
    centroid = geometry.centroid
    h3_centroid = h3.geo_to_h3(centroid.y, centroid.x, centroid_res)

    # Full coverage using polyfill
    h3_polyfill = list(h3.polyfill(geometry.__geo_interface__, polyfill_res))

    return h3_centroid, h3_polyfill

In [ ]:
test_prev = prev_month_blobs.copy()
test_curr = curr_month_blobs.copy()

for gdf in [test_prev, test_curr]:
    gdf['h3_centroid'] = gdf['geometry'].apply(lambda geom: compute_h3_indices(geom, 6, 9))

In [899]:
print(type(prev_month_blobs))
prev_month_blobs.head(3)
# prev_month_blobs = gpd.read_parquet('data/prev_month_blobs.parquet')
# curr_month_blobs = gpd.read_parquet('data/curr_month_blobs.parquet')

<class 'geopandas.geodataframe.GeoDataFrame'>


,geometry,id,geohash
0,"POLYGON ((-81.10504 26.01885, -81.04984 26.018...",6cc95c8a-cbaf-4993-a9ff-4e890c7e45f0,dhwk5h
1,"POLYGON ((-80.91675 26.04190, -80.86358 26.041...",a0fbead2-d8dc-4e05-b364-ab89d020fbfb,dhwkpq
2,"POLYGON ((-80.73525 26.20304, -80.71903 26.203...",b4c64915-35eb-4d74-a211-ce41ad8f21f8,dhwt55


# Notes on original process

geohash polygon into a larger region
match blobs using multiprocessing (mp.Process) and geopandas
find blobs from previous month that do not have a match

# 1 imports and setup

```python
import os, sys, traceback
import argparse
import cv2
import time
import numpy as np
import pandas as pd
import geopandas as gpd
from skimage import io, color, measure
from tqdm import tqdm
from shapely import wkt
from shapely.geometry import Point, Polygon
from shapely.validation import make_valid
import PIL
from PIL import Image, ImageDraw, ImageEnhance
import uuid
import multiprocessing as mp
from datetime import datetime
from sqlalchemy import or_
from sqlmodel import SQLModel, Session, create_engine, select
from pathlib import Path
```

## Key takeaways
- Uses Pandas and GeoPandas for data processing.
- Uses Shapely for geospatial geometry operations.
- Uses multiprocessing (mp) to parallelize the blob-matching process.
- Uses SQLAlchemy and SQLModel for database operations.

# 2 High-level script overview
The script is designed to match blobs (spatial objects) between two months and classify them into different business categories.

General Workflow
1. Fetch blobs from the previous and current months.
1. Convert the region of interest (state, city, county, or geohash) into a list of geohashes.
1. Parallelized Matching Process:
1. Process multiple geohashes at once using multiprocessing.
1. Find corresponding blobs from previous months for each geohash.
1. Classify blobs based on construction stage progression.
1. Identify blobs that are missing in the current month and "impute" them.
1. Save results into the database.

# 3 Blob classification logic

# 4 Blob matching

- Matches blobs between months using polygon intersections.
- Uses Shapely to validate geometries and check for overlaps.
- Returns matched blob IDs.

## Multiprocessing Optimization
The script parallelizes blob matching by:
- Uses multiprocessing (mp.Process) to divide the dataset into smaller batches.
- Each batch of blobs is processed in parallel.
- Reduces runtime compared to a single-threaded approach.

# 5 Handling unmatched blobs
- Finds blobs from the previous month that do not have a match in the current month.
- These blobs are "imputed", meaning they are carried over into the new month.

# 6 Main class to orchestrate process
- Loads previous and current month blobs.
- Converts input regions (state, city, county) into geohashes.

# Summary
- Parallel processing is used to match blobs.
- Blobs are classified based on construction stages.
- Unmatched blobs are imputed for continuity.
- The script writes results to a database.